# Find Nearest ATM

Use this notebook to enrich the DSK office list with geographic coordinates so that follow-up nearest-ATM lookups can rely on latitude/longitude instead of free-form addresses. The steps below load the office JSON, geocode each address, and persist the results.

> ⚠️ Geocoding calls an external (or locally hosted) Nominatim service. Respect the service usage policy, throttle requests, and prefer a self-hosted instance when possible.

In [59]:
# Optional: install dependencies in the active kernel (uncomment if needed)
# %pip install geopy pandas


In [60]:

import json
from pathlib import Path
from typing import Any, Dict, Iterable, Optional, Tuple

import re
import time

import pandas as pd

try:
    from geopy.geocoders import Nominatim
    from geopy.extra.rate_limiter import RateLimiter
except ModuleNotFoundError as exc:
    raise ModuleNotFoundError('Install geopy in this environment: `pip install geopy`.') from exc

try:
    from unidecode import unidecode
except ModuleNotFoundError:
    unidecode = None

try:
    import requests
except ModuleNotFoundError as exc:
    raise ModuleNotFoundError('Install requests in this environment: `pip install requests`.') from exc


In [61]:
DATA_PATH = Path('dsk_offices_parsed.json')

with DATA_PATH.open(encoding='utf-8') as fp:
    offices_payload = json.load(fp)

entries = offices_payload['entries']
offices_df = pd.DataFrame(entries)
offices_df.head()


,otp_id,city,office_name,type,address_line,hours,source
0,315,Варна,ВЛАДИСЛАВ,ФЦ,"бул. ""Вл. Варненчик"" 92",08:45 - 13:00; 13:30 - 16:45,Branches.pdf
1,337,Варна,ПРЕСЛАВ,ФЦ,"ул. ""Преслав"" 69",08:45 - 13:00; 13:30 - 16:45,Branches.pdf
2,2404,Каварна,КАВАРНА,Клон,"ул. ""Добротица"" № 25",08:30 - 17:00,Branches.pdf
3,704,Севлиево,СЕВЛИЕВО,Клон,"пл. ""Свобода"" № 9",08:30 - 17:00,Branches.pdf
4,401,Велико,Търново ВЕЛИКО ТЪРНОВО,ФЦ,"ул. ""Цар Освободител"" № 3",08:30 - 17:00,Branches.pdf


Configure the geocoding client. Toggle `USE_STRUCTURED_QUERIES`/`USE_VIEWBOX` for Nominatim tweaks and enable `USE_PHOTON_FALLBACK` or `USE_OPEN_METEO_FALLBACK` when you want secondary services to retry unresolved addresses. Adjust delays if you control the backend services.


In [62]:

GEOCODER_DOMAIN = 'nominatim.openstreetmap.org'  # set to 'localhost:8080' when using a local instance
GEOCODER_SCHEME = 'https' if GEOCODER_DOMAIN.endswith('openstreetmap.org') else 'http'
USER_AGENT = 'dsk-office-geocoder'
MIN_DELAY_SECONDS = 1.1  # follow the public Nominatim usage policy
REQUEST_TIMEOUT = 10
GEOCODER_KWARGS = {
    'country_codes': 'bg',
    'addressdetails': True,
    'language': 'bg',
}
USE_STRUCTURED_QUERIES = True
USE_VIEWBOX = True
VIEWBOX_PADDING_DEGREES = 0.01

USE_PHOTON_FALLBACK = True
PHOTON_ENDPOINT = 'https://photon.komoot.io/api/'
PHOTON_RESULT_LIMIT = 10
PHOTON_DELAY_SECONDS = 1.0

USE_OPEN_METEO_FALLBACK = True
OPEN_METEO_ENDPOINT = 'https://geocoding-api.open-meteo.com/v1/search'
OPEN_METEO_RESULT_LIMIT = 10
OPEN_METEO_DELAY_SECONDS = 1.0

geolocator = Nominatim(
    user_agent=USER_AGENT,
    domain=GEOCODER_DOMAIN,
    scheme=GEOCODER_SCHEME,
    timeout=REQUEST_TIMEOUT,
)
geocode = RateLimiter(
    geolocator.geocode,
    min_delay_seconds=MIN_DELAY_SECONDS,
    swallow_exceptions=False,
)

REQUEST_SESSION = requests.Session()


In [63]:

PUNCTUATION_TO_SPACE = str.maketrans({
    '"': ' ',
    '„': ' ',
    '“': ' ',
    '”': ' ',
    '«': ' ',
    '»': ' ',
})
ADDRESS_REPLACEMENTS = {
    'бул.': 'булевард',
    'ул.': 'улица',
    'ж.к.': 'жилищен комплекс',
    'жк.': 'жилищен комплекс',
    'кв.': 'квартал',
    'пл.': 'площад',
}
COUNTRY_NAME = 'Bulgaria'
PREFERRED_OSM_CLASSES = {
    'amenity',
    'building',
    'highway',
    'office',
    'shop',
    'tourism',
}
PREFERRED_OSM_TYPES = {
    'apartments',
    'bank',
    'building',
    'commercial',
    'detached',
    'house',
    'mall',
    'office',
    'public',
    'retail',
    'supermarket',
    'yes',
}
PLACE_TYPES_TO_AVOID = {'city', 'town', 'village', 'county', 'state', 'country'}
MAX_GEO_RESULTS = 5
MIN_ACCEPTED_SCORE = 0
CITY_VIEWBOX_CACHE: Dict[str, Tuple[Tuple[float, float], Tuple[float, float]]] = {}
PHOTON_CACHE: Dict[str, Dict[str, Any]] = {}
OPEN_METEO_CACHE: Dict[str, Dict[str, Any]] = {}
PHOTON_THROTTLE = {'value': 0.0}
OPEN_METEO_THROTTLE = {'value': 0.0}

def normalize_address_line(address: str) -> str:
    if not address:
        return ''
    normalized = address.translate(PUNCTUATION_TO_SPACE)
    normalized = normalized.replace('№', 'номер ')
    for src, dst in ADDRESS_REPLACEMENTS.items():
        normalized = re.sub(rf'\b{re.escape(src)}\s*', f'{dst} ', normalized, flags=re.IGNORECASE)
    normalized = re.sub(r'\s+', ' ', normalized).strip(' ,')
    return normalized

def maybe_transliterate(text: str) -> str:
    if not text:
        return ''
    if unidecode is None:
        return text
    return unidecode(text)

def generate_queries(entry: Dict[str, str]) -> Iterable[str]:
    address = (entry.get('address_line') or '').strip()
    city = (entry.get('city') or '').strip()
    raw_parts = [address, city, COUNTRY_NAME]
    normalized_address = normalize_address_line(address)
    normalized_parts = [normalized_address, city, COUNTRY_NAME]
    candidates = [
        ', '.join([part for part in raw_parts if part]),
        ', '.join([part for part in normalized_parts if part]),
        ', '.join([part for part in [city, COUNTRY_NAME] if part]),
        address,
        normalized_address,
    ]
    romanized = maybe_transliterate(', '.join([part for part in raw_parts if part]))
    if romanized and romanized not in candidates:
        candidates.append(romanized)
    romanized_normalized = maybe_transliterate(', '.join([part for part in normalized_parts if part]))
    if romanized_normalized and romanized_normalized not in candidates:
        candidates.append(romanized_normalized)
    unique_queries = []
    seen = set()
    for candidate in candidates:
        if not candidate:
            continue
        candidate = re.sub(r'\s+', ' ', candidate).strip(' ,')
        key = candidate.lower()
        if key in seen:
            continue
        seen.add(key)
        unique_queries.append(candidate)
    return unique_queries

def build_structured_queries(entry: Dict[str, str]) -> Iterable[Dict[str, str]]:
    address = (entry.get('address_line') or '').strip()
    city = (entry.get('city') or '').strip()
    normalized = normalize_address_line(address)
    candidates = [address, normalized, maybe_transliterate(address), maybe_transliterate(normalized)]
    unique_streets = []
    seen = set()
    for street in candidates:
        cleaned = (street or '').strip()
        if not cleaned:
            continue
        key = cleaned.lower()
        if key in seen:
            continue
        seen.add(key)
        unique_streets.append(cleaned)
    queries = []
    for street in unique_streets:
        parts = {'country': COUNTRY_NAME}
        if city:
            parts['city'] = city
        if street:
            parts['street'] = street
        queries.append(parts)
    if city and not queries:
        queries.append({'country': COUNTRY_NAME, 'city': city})
    return queries

def parse_float(value: Any) -> float:
    try:
        return float(value)
    except (TypeError, ValueError):
        return 0.0

def score_location(location) -> Tuple[int, float]:
    raw = location.raw or {}
    osm_class = raw.get('class') or ''
    osm_type = raw.get('type') or ''
    address = raw.get('address') or {}
    score = 0
    if osm_class in PREFERRED_OSM_CLASSES:
        score += 3
    if osm_type in PREFERRED_OSM_TYPES:
        score += 3
    if address.get('house_number'):
        score += 3
    if address.get('road'):
        score += 1
    if osm_class == 'place' and osm_type in PLACE_TYPES_TO_AVOID:
        score -= 4
    if osm_class == 'boundary':
        score -= 3
    return score, parse_float(raw.get('importance'))

def select_best_location(locations):
    best_location = None
    best_score = (-float('inf'), -float('inf'))
    for location in locations:
        score = score_location(location)
        if score > best_score:
            best_score = score
            best_location = location
    return best_location, best_score

def _normalize_viewbox(viewbox):
    if not viewbox:
        return None
    south, west = viewbox[0]
    north, east = viewbox[1]
    return (
        (round(south, 6), round(west, 6)),
        (round(north, 6), round(east, 6)),
    )

def make_cache_key(query: Any, structured: bool, viewbox, bounded: bool):
    if structured:
        items = tuple(sorted((k, v) for k, v in query.items()))
    else:
        items = query
    viewbox_key = None
    if viewbox:
        viewbox_key = tuple(round(coord, 6) for coord in (viewbox[0][0], viewbox[0][1], viewbox[1][0], viewbox[1][1]))
    return (structured, items, viewbox_key, bool(bounded))

def parse_bounding_box(raw_box) -> Optional[Tuple[float, float, float, float]]:
    if not raw_box:
        return None
    try:
        south, north, west, east = map(float, raw_box)
    except (TypeError, ValueError):
        return None
    return south, north, west, east

def build_viewbox(bbox: Tuple[float, float, float, float]) -> Tuple[Tuple[float, float], Tuple[float, float]]:
    south, north, west, east = bbox
    south -= VIEWBOX_PADDING_DEGREES
    north += VIEWBOX_PADDING_DEGREES
    west -= VIEWBOX_PADDING_DEGREES
    east += VIEWBOX_PADDING_DEGREES
    south = max(-90.0, south)
    north = min(90.0, north)
    west = max(-180.0, west)
    east = min(180.0, east)
    return (south, west), (north, east)

def throttle_call(state: Dict[str, float], min_delay: float) -> None:
    now = time.monotonic()
    elapsed = now - state.get('value', 0.0)
    if elapsed < min_delay:
        time.sleep(min_delay - elapsed)
    state['value'] = time.monotonic()

def get_city_viewbox(city: str):
    if not USE_VIEWBOX or not city:
        return None
    key = city.strip().lower()
    if not key:
        return None
    if key in CITY_VIEWBOX_CACHE:
        return CITY_VIEWBOX_CACHE[key]
    try:
        location = geocode({'city': city, 'country': COUNTRY_NAME}, exactly_one=True, limit=1, **GEOCODER_KWARGS)
    except Exception:
        CITY_VIEWBOX_CACHE[key] = None
        return None
    if not location or not getattr(location, 'raw', None):
        CITY_VIEWBOX_CACHE[key] = None
        return None
    bbox = parse_bounding_box(location.raw.get('boundingbox'))
    if not bbox:
        CITY_VIEWBOX_CACHE[key] = None
        return None
    viewbox = build_viewbox(bbox)
    CITY_VIEWBOX_CACHE[key] = viewbox
    return viewbox

def geocode_with_options(query, *, structured: bool, viewbox=None, bounded: bool = False):
    params = dict(GEOCODER_KWARGS)
    if viewbox:
        params['viewbox'] = viewbox
        if bounded:
            params['bounded'] = 1
    try:
        results = geocode(query, exactly_one=False, limit=MAX_GEO_RESULTS, **params)
    except Exception:
        return {}
    if not results:
        return {}
    if not isinstance(results, (list, tuple)):
        results = [results]
    best_location, (quality, importance) = select_best_location(results)
    if not best_location:
        return {}
    raw = best_location.raw or {}
    return {
        'latitude': best_location.latitude,
        'longitude': best_location.longitude,
        'address': best_location.address or '',
        'osm_type': raw.get('type'),
        'osm_class': raw.get('class'),
        'place_id': raw.get('place_id'),
        'place_rank': raw.get('place_rank'),
        'importance': importance,
        'quality': quality,
        '_structured': structured,
        '_bounded': bool(bounded),
        '_viewbox': _normalize_viewbox(viewbox),
    }

def _bbox_to_photon(viewbox) -> Optional[str]:
    if not viewbox:
        return None
    (south, west), (north, east) = viewbox
    return f"{west},{south},{east},{north}"

def score_photon_feature(feature: Dict[str, Any]) -> Tuple[int, float]:
    props = feature.get('properties', {})
    score = 0
    if props.get('housenumber'):
        score += 3
    if props.get('street'):
        score += 1
    osm_value = props.get('osm_value') or ''
    if osm_value in PREFERRED_OSM_TYPES:
        score += 3
    osm_key = props.get('osm_key') or ''
    if osm_key in PREFERRED_OSM_CLASSES:
        score += 2
    if props.get('type') in ('city', 'county', 'state', 'country'):
        score -= 4
    importance = parse_float(props.get('importance'))
    return score, importance

def photon_geocode(query: str, city: str, viewbox) -> Dict[str, Any]:
    if not USE_PHOTON_FALLBACK or not query:
        return {}
    cache_key = (query, city or '', _normalize_viewbox(viewbox))
    if cache_key in PHOTON_CACHE:
        return PHOTON_CACHE[cache_key]
    params = {
        'q': query,
        'limit': PHOTON_RESULT_LIMIT,
        'lang': 'bg',
    }
    bbox = _bbox_to_photon(viewbox)
    if bbox:
        params['bbox'] = bbox
    try:
        throttle_call(PHOTON_THROTTLE, PHOTON_DELAY_SECONDS)
        response = REQUEST_SESSION.get(PHOTON_ENDPOINT, params=params, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()
        payload = response.json()
    except Exception:
        PHOTON_CACHE[cache_key] = {}
        return {}
    features = payload.get('features') or []
    best_feature = None
    best_score = (-float('inf'), -float('inf'))
    for feature in features:
        score = score_photon_feature(feature)
        if score > best_score:
            best_score = score
            best_feature = feature
    if not best_feature:
        PHOTON_CACHE[cache_key] = {}
        return {}
    geometry = best_feature.get('geometry') or {}
    coords = geometry.get('coordinates') or []
    if len(coords) < 2:
        PHOTON_CACHE[cache_key] = {}
        return {}
    props = best_feature.get('properties') or {}
    result = {
        'latitude': coords[1],
        'longitude': coords[0],
        'address': props.get('name') or props.get('street') or query,
        'osm_type': props.get('osm_value'),
        'osm_class': props.get('osm_key'),
        'place_id': props.get('osm_id'),
        'place_rank': props.get('type'),
        'importance': best_score[1],
        'quality': best_score[0],
        'source': 'photon',
    }
    PHOTON_CACHE[cache_key] = result
    return result

def score_open_meteo(result: Dict[str, Any]) -> Tuple[int, float]:
    score = 0
    if result.get('feature_code') in ('PPLA', 'PPLC'):
        score += 2
    if result.get('population', 0) and result['population'] > 0:
        score += 1
    importance = 0.0
    return score, importance

def open_meteo_geocode(city: str) -> Dict[str, Any]:
    if not USE_OPEN_METEO_FALLBACK or not city:
        return {}
    key = city.strip().lower()
    if key in OPEN_METEO_CACHE:
        return OPEN_METEO_CACHE[key]
    params = {
        'name': city,
        'count': OPEN_METEO_RESULT_LIMIT,
        'language': 'bg',
        'format': 'json',
    }
    try:
        throttle_call(OPEN_METEO_THROTTLE, OPEN_METEO_DELAY_SECONDS)
        response = REQUEST_SESSION.get(OPEN_METEO_ENDPOINT, params=params, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()
        payload = response.json()
    except Exception:
        OPEN_METEO_CACHE[key] = {}
        return {}
    results = payload.get('results') or []
    best_result = None
    best_score = (-float('inf'), -float('inf'))
    for item in results:
        score = score_open_meteo(item)
        if score > best_score:
            best_score = score
            best_result = item
    if not best_result:
        OPEN_METEO_CACHE[key] = {}
        return {}
    result = {
        'latitude': best_result.get('latitude'),
        'longitude': best_result.get('longitude'),
        'address': best_result.get('name') or city,
        'osm_type': best_result.get('feature_code'),
        'osm_class': 'place',
        'place_id': best_result.get('id'),
        'place_rank': best_result.get('feature_code'),
        'importance': 0.0,
        'quality': best_score[0],
        'source': 'open-meteo',
    }
    OPEN_METEO_CACHE[key] = result
    return result


In [64]:

geocode_cache: Dict[tuple, Dict[str, Any]] = {}
enriched_entries = []
for entry in entries:
    candidates = list(generate_queries(entry))
    structured_queries = list(build_structured_queries(entry)) if USE_STRUCTURED_QUERIES else []
    city = (entry.get('city') or '').strip()
    viewbox = get_city_viewbox(city)
    attempts = []
    seen_attempt_keys = set()

    def add_attempt(structured: bool, query, use_viewbox: bool, bounded: bool):
        vb = viewbox if use_viewbox else None
        cache_key = make_cache_key(query, structured, vb, bounded)
        if cache_key in seen_attempt_keys:
            return
        seen_attempt_keys.add(cache_key)
        attempts.append((structured, query, vb, bounded, cache_key))

    for structured_query in structured_queries:
        if viewbox:
            add_attempt(True, structured_query, True, True)
        add_attempt(True, structured_query, False, False)

    for candidate in candidates:
        if viewbox:
            add_attempt(False, candidate, True, True)
        add_attempt(False, candidate, False, False)

    best_result = None
    best_context = None
    best_quality = float('-inf')

    for structured, query, vb, bounded, cache_key in attempts:
        result = geocode_cache.get(cache_key)
        if result is None:
            result = geocode_with_options(query, structured=structured, viewbox=vb, bounded=bounded)
            geocode_cache[cache_key] = result
        if not result:
            continue
        quality = result.get('quality', float('-inf'))
        if quality > best_quality:
            best_quality = quality
            best_result = result
            best_context = {
                'structured': structured,
                'bounded': bool(bounded),
                'viewbox': result.get('_viewbox'),
                'query': query,
                'source': 'nominatim',
            }

    if not best_result:
        for candidate in candidates:
            fallback = photon_geocode(candidate, city, viewbox)
            if fallback:
                best_result = fallback
                best_quality = fallback.get('quality', float('-inf'))
                best_context = {
                    'structured': False,
                    'bounded': bool(viewbox),
                    'viewbox': _normalize_viewbox(viewbox),
                    'query': candidate,
                    'source': 'photon',
                }
                break

    if not best_result:
        fallback = open_meteo_geocode(city)
        if fallback:
            best_result = fallback
            best_quality = fallback.get('quality', float('-inf'))
            best_context = {
                'structured': True,
                'bounded': False,
                'viewbox': None,
                'query': {'city': city, 'country': COUNTRY_NAME},
                'source': 'open-meteo',
            }

    enriched_entry = dict(entry)
    enriched_entry['geocode_candidates'] = candidates
    if structured_queries:
        enriched_entry['geocode_structured_candidates'] = structured_queries

    if best_result and best_quality >= MIN_ACCEPTED_SCORE:
        enriched_entry['latitude'] = best_result.get('latitude')
        enriched_entry['longitude'] = best_result.get('longitude')
        enriched_entry['geocoded_address'] = best_result.get('address')
        enriched_entry['geocode_osm_type'] = best_result.get('osm_type')
        enriched_entry['geocode_osm_class'] = best_result.get('osm_class')
        enriched_entry['geocode_place_id'] = best_result.get('place_id')
        enriched_entry['geocode_place_rank'] = best_result.get('place_rank')
        enriched_entry['geocode_quality'] = best_result.get('quality')
        enriched_entry['geocode_importance'] = best_result.get('importance')
        if best_context:
            enriched_entry['geocode_query'] = best_context.get('query')
            enriched_entry['geocode_context'] = {
                'structured': best_context.get('structured'),
                'bounded': best_context.get('bounded'),
                'viewbox': best_context.get('viewbox'),
                'source': best_context.get('source'),
            }
            enriched_entry['geocode_source'] = best_context.get('source')
    else:
        enriched_entry['latitude'] = None
        enriched_entry['longitude'] = None

    enriched_entries.append(enriched_entry)

enriched_df = pd.DataFrame(enriched_entries)
enriched_df[['office_name', 'latitude', 'longitude', 'geocode_quality', 'geocode_query']].head()


RateLimiter caught an error, retrying (0/2 tries). Called with (*('булевард Цариградско шосе номер 115 З, София, Bulgaria',), **{'exactly_one': False, 'limit': 5, 'country_codes': 'bg', 'addressdetails': True, 'language': 'bg', 'viewbox': ((42.547985100000005, 23.157996999999998), (42.874302, 23.6315817)), 'bounded': 1}).
Traceback (most recent call last):
  File "/Users/ivanangelov/Documents/GitHub/nominatim-openstreetmap/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ivanangelov/Documents/GitHub/nominatim-openstreetmap/.venv/lib/python3.12/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socket.py", line 963, in

,office_name,latitude,longitude,geocode_quality,geocode_query
0,ВЛАДИСЛАВ,NaN,NaN,NaN,NaN
1,ПРЕСЛАВ,41.651078,25.373972,4.0,"ул. ""Преслав"" 69"
2,КАВАРНА,NaN,NaN,NaN,NaN
3,СЕВЛИЕВО,43.026251,25.104217,4.0,"{'country': 'Bulgaria', 'city': 'Севлиево', 's..."
4,Търново ВЕЛИКО ТЪРНОВО,43.207384,27.924131,4.0,"ул. ""Цар Освободител"" № 3"


In [66]:

    missing = enriched_df[enriched_df['latitude'].isna()]
    print(f'Missing coordinates for {len(missing)} of {len(enriched_df)} offices.')
    if not missing.empty:
        display(missing[['office_name', 'city', 'address_line', 'geocode_quality', 'geocode_context']].head())

    duplicates = (
        enriched_df.dropna(subset=['latitude', 'longitude'])
        .groupby(['latitude', 'longitude'])
        .filter(lambda group: len(group) > 1)
        .copy()
    )
    print(f'Duplicate coordinate pairs: {len(duplicates)} entries across {duplicates.groupby(["latitude", "longitude"]).ngroups} groups.')
    if not duplicates.empty:
        display(duplicates[['office_name', 'city', 'address_line', 'geocode_quality', 'geocode_query', 'geocode_context']])

    if 'geocode_source' in enriched_df.columns:
        print('
Resolved coordinate sources:')
        source_counts = (
            enriched_df['geocode_source']
            .fillna('none')
            .value_counts()
            .rename_axis('source')
            .reset_index(name='count')
        )
        display(source_counts)


IndentationError: unexpected indent (4093013718.py, line 1)

Persist the enriched payload. By default we keep the original file untouched and write a sibling file. Change `OUTPUT_PATH` to `DATA_PATH` if you want to overwrite in place once you are happy with the results.

In [57]:
OUTPUT_PATH = Path('dsk_offices_parsed_with_coords.json')

offices_payload['entries'] = enriched_entries
with OUTPUT_PATH.open('w', encoding='utf-8') as fp:
    json.dump(offices_payload, fp, ensure_ascii=False, indent=2)
print(f'Wrote {len(enriched_entries)} entries with coordinates to {OUTPUT_PATH}')


Wrote 37 entries with coordinates to dsk_offices_parsed_with_coords.json


In [58]:
enriched_df[["city", "address_line", "latitude", "longitude"]].query("city == 'София'")

,city,address_line,latitude,longitude
19,София,"ул. ""Дебър"" № 1",NaN,NaN
20,София,"бул. ""Ал. Стамболийски"" 73",44.114570,27.263917
21,София,"ул. ""Княз Александър I"" № 6",42.695017,23.324281
22,София,"ул. ""Московска"" № 19",NaN,NaN
23,София,"ж.к. ""Илинден"" бл. 129-130 A",NaN,NaN
34,София,"бул. ""Ситняково"" № 48",42.689053,23.353111
35,София,"бул. „Цариградско шосе“ № 115 ""З“",42.634281,23.439167
36,София,"бул. ""Ал. Стамболийски"" № 101",44.114570,27.263917


# Find closest ATM